# Landcover consensus

In [21]:
using SpeciesDistributionToolkit
using CairoMakie
CairoMakie.activate!(px_per_unit = 6.0)

In [ ]:
place = SpeciesDistributionToolkit.gadm("PRY")
extent = SpeciesDistributionToolkit.boundingbox(place)

In [ ]:
provider = RasterData(EarthEnv, LandCover)
luclasses = SDMLayer{UInt8}[SDMLayer(provider; layer=l, extent...) for l in layers(provider)]
mask!(luclasses, place)

The `mosaic` function will apply its first input (a function operating on a vector and returning an atomic value) to all cells with the same coordinate in its second argument, a vector of layers. In this case, we use the `argmax` function to identify which layer represents the most locally abundant land cover class:

In [ ]:
consensus = mosaic(argmax, luclasses)

In [ ]:
landcover_colors = ["#117733", "#668822", "#99BB55", "#55aa22", "#ddcc66", "#aaddcc", "#44aa88", "#88bbaa", "#bb0011", :aqua, "#FFEE88", "#5566AA"]

In [ ]:
#| label: fig-landcover-consensus
#| fig-cap: "yeah"
fig = Figure()
panel = Axis(fig[1, 1]; aspect = DataAspect())
heatmap!(panel, consensus; colormap = landcover_colors)
lines!(panel, place[1].geometry, color=:black)
tightlimits!(panel)
hidespines!(panel)
hidedecorations!(panel)
Legend(
    fig[2, 1],
    [PolyElement(; color = landcover_colors[i]) for i in eachindex(landcover_colors)],
    layers(provider)[1:length(luclasses)];
    orientation = :horizontal,
    nbanks = 4,
    framevisible=false
)
fig